#  기계학습 실습 Week 6b (CNN regression, 사진 나이 맞추기)

2024.10.17.<br>

서강대 경제대학 양현주 (hyang@sogang.ac.kr)<br><br>

This notebook uses CNN to do age prediction.<br><br>

Data source for face images: <br>
https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/ <br><br>

Codes are based on: <br>
https://medium.com/analytics-vidhya/fastai-image-regression-age-prediction-based-on-image-68294d34f2ed <br><br>

For image augmentation, refer to: <br>
https://github.com/fastai/fastbook/blob/master/02_production.ipynb <br><br>

For AI and ethics, refer to: <br>
https://github.com/fastai/fastbook/blob/master/03_ethics.ipynb <br><br>

## 평가기준
### 분류
- accuracy
- recall / precision <br><br>
### 회귀
- MAE, MSE, RMSE(root MSE)

# 1. Install and import libraries

In [ ]:
import fastai
print(fastai.__version__)

from fastai.vision.all import *
#from fastai.text.all import *
#from fastai.collab import *
#from fastai.tabular.all import *

from matplotlib.pyplot import imshow

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import imshow
from google.colab import files

# 2. Download file

In [ ]:
# upload imdb_crop_sample_5k.zip

# if using dropbox
#!wget -O imdb_crop_sample_5k.zip YOUR-OWN-DROPBOX-LINK

In [ ]:
%%capture

!unzip imdb_crop_sample_5k.zip -d faces

In [ ]:
!ls

# 3. Prepare image file path + label dataframe

## 3.1 grab image file paths

In [ ]:
# *** CHANGE directory and file extension HERE ***

where_and_what_to_search = 'faces/*.jpg' # YOUR IMAGE SUBFOLDER / IMAGE FILE EXTENSION

# grab image file paths

import glob
img_full_path = pd.Series(glob.glob(where_and_what_to_search), name='my_file_path')
img_nm = pd.Series(img_full_path.str.split(pat="/").str[1], name='file_nm')

df_imdb = pd.concat([img_full_path, img_nm], axis=1)
df_imdb

## 3.2 Extract the year of photo taken & year of birth from file names

In [ ]:
# extract photos taken from file names from 'file_nm'

df_imdb['photo_taken'] = df_imdb.file_nm.str.extract(r'[\_](\d\d\d\d)[\.][j][p][g]$')

In [ ]:
# extract year of birth from 'file_nm'

df_imdb['year_of_birth'] = df_imdb.file_nm.str.extract(r'[\_](\d\d\d\d)[-]')

In [ ]:
df_imdb

## 3.3 Check missing labels

In [ ]:
# check if missing variable
df_imdb.isnull().sum()

In [ ]:
# delete NaN sample
df_imdb = df_imdb.replace([np.inf, -np.inf], np.nan)
df_imdb.dropna(inplace=True)

In [ ]:
# check if missing variable
df_imdb.isnull().sum()

In [ ]:
df_imdb.shape

## 3.4 Calculate age

In [ ]:
df_imdb.dtypes

In [ ]:
# convert columns related to years to int64 dtype
df_imdb = df_imdb.astype({"photo_taken": int, "year_of_birth": int})

In [ ]:
df_imdb.dtypes

In [ ]:
# calculate age
df_imdb['age'] = df_imdb['photo_taken'] - df_imdb['year_of_birth']

# some guys seem to be greater than 100. some of these are paintings. remove these old guys
df_imdb = df_imdb[df_imdb['age'] <= 100]

# some guys seem to be unborn in the data set
df_imdb = df_imdb[df_imdb['age'] > 0]

In [ ]:
df_imdb.shape

In [ ]:
df_imdb

In [ ]:
df_imdb['age'].hist()

# 4. Prepare data for CNN

In [ ]:
# data block settings

my_random_seed = 128
my_batch_size = 64

In [ ]:
from fastai.vision.data import ImageDataLoaders

In [ ]:
df_imdb_simple = df_imdb[['my_file_path','age']]
df_imdb_simple

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x=ColReader('my_file_path'),
    get_y=ColReader('age'),
    splitter=RandomSplitter(valid_pct=0.2, seed=my_random_seed),
    item_tfms=Resize(128)
).dataloaders(df_imdb_simple)

In [ ]:
len(dls.train_ds), len(dls.valid_ds)

In [ ]:
# show image examples

dls.show_batch(max_n=16, nrows=2)

# 5. Train CNN model

In [ ]:
learn = vision_learner(dls, resnet18, metrics=rmse).to_fp16() # resnet 18, 34, 50, 101, 152
learn.fine_tune(5)

In [ ]:
learn.show_results()

# 6. Test using your own image

In [ ]:
# upload file to colab and set file path
img_fpath = './mona_lisa.jpg'

In [ ]:
# show image

from IPython.display import display
from PIL import Image

img = Image.open(img_fpath)

display(img)

In [ ]:
learn.predict(img)

# 7. Image augmentation

## 7.1 Random Resized Crop

In [ ]:
dls_aug = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x=ColReader('my_file_path'),
    get_y=ColReader('age'),
    splitter=RandomSplitter(valid_pct=0.2, seed=my_random_seed),
    item_tfms=RandomResizedCrop(128, min_scale=0.7)
).dataloaders(df_imdb_simple)

In [ ]:
dls_aug.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
learn2 = vision_learner(dls_aug, resnet18, metrics=rmse).to_fp16() # resnet 18, 34, 50, 101, 152
learn2.fine_tune(5)

## 7.2 aug_transforms

In [ ]:
dls_aug2 = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x=ColReader('my_file_path'),
    get_y=ColReader('age'),
    splitter=RandomSplitter(valid_pct=0.2, seed=my_random_seed),
    item_tfms=Resize(128),
    batch_tfms=aug_transforms(mult=2)
).dataloaders(df_imdb_simple)

In [ ]:
dls_aug2.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
learn3 = vision_learner(dls_aug2, resnet18, metrics=rmse).to_fp16() # resnet 18, 34, 50, 101, 152
learn3.fine_tune(5)

# 8. Early stopping

In [ ]:
learn4 = vision_learner(dls_aug2, resnet18, metrics=rmse).to_fp16() # resnet 18, 34, 50, 101, 152
learn4.path = Path('./')
learn4.fine_tune(50, cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=10),SaveModelCallback(monitor='valid_loss')])

In [ ]:
learn4.validate()

In [ ]:
learn4.show_results()